In [1]:
import pandas as pd
import time
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from datetime import datetime
from datetime import datetime,timedelta
import mysql.connector
import pandas as pd
import numpy as np
import sqlalchemy
import pymysql
import mysql


In [2]:
df = pd.read_csv('In Count Data.csv')
df["DATE"] = pd.to_datetime([datetime.strptime(each,"%d-%m-%Y") for each in df["DATE"]],format = "%d-%m-%Y")
df["DATE"].head()

0   2020-06-18
1   2020-06-18
2   2020-06-18
3   2020-06-18
4   2020-06-18
Name: DATE, dtype: datetime64[ns]

In [3]:
a = df[["Employee_ID","Employee_Name","Department","DATE","IN TIME"]]
dd = df.groupby(["Employee_ID","DATE"])["Employee_Name","Department","IN TIME"].min()
in_time = dd.reset_index()
in_time.head()
# a
# print(type(a))
# dd

,Employee_ID,DATE,Employee_Name,Department,IN TIME
0,281,2020-10-12,R.Kishore Kumar,Multimedia,11:29
1,281,2020-11-02,R.Kishore Kumar,Multimedia,15:49
2,336,2020-10-12,K.V.Sudhakara Reddy,Citadel,11:28
3,336,2020-11-02,K.V.Sudhakara Reddy,Citadel,15:48
4,483,2020-10-12,K.Sreedhar,Citadel,11:28


In [4]:
df1 = pd.read_csv('In Count Data2.csv')
df1

# df1["DATE"] = pd.to_datetime([datetime.strptime(each,"%d/%m/%Y") for each in df1["DATE"]],format = "%d/%m/%Y")
df1["DATE"]= df1["DATE"].astype('datetime64')
df1["DATE"].head()

0   2020-06-18
1   2020-06-18
2   2020-06-18
3   2020-06-18
4   2020-06-18
Name: DATE, dtype: datetime64[ns]

In [5]:
# Out Time min()
dfff = df1[["Employee_ID","Employee_Name","Department","DATE","OUT TIME"]]
ddd = dfff.groupby(["Employee_ID","DATE"])["Employee_Name","Department","OUT TIME"].max()
out_time = ddd.reset_index()
out_time["DATE"].head()

0   2020-06-18
1   2020-07-09
2   2020-07-09
3   2020-06-18
4   2020-06-18
Name: DATE, dtype: datetime64[ns]

In [6]:
atn = pd.merge(in_time, out_time, on = ["Employee_ID","Employee_Name","Department","DATE"], how = "outer")
attendence_fnl = atn.sort_values(by=["DATE"], axis=0,ascending = False, inplace=False, kind='quicksort', na_position='last')
td = attendence_fnl[attendence_fnl["DATE"] == datetime.now().strftime("%d-%m-%Y")]
td
attendence_fnl.head()

,Employee_ID,DATE,Employee_Name,Department,IN TIME,OUT TIME
123,2258,2021-02-16,Kobbana Vasanth Rao,Data Analytics,17:02,NaN
68,2108,2021-02-16,Parisa Bala Venkata Sai Raghava,Data Analytics,17:01,NaN
141,6015,2021-02-16,Karkarlapudi Vineeth Varma,Data Analytics,17:04,NaN
53,2065,2020-11-02,Venkata Amaranath Reddy Nandyala,OTC,15:50,NaN
51,2056,2020-11-02,Sanivarapu Krishnaveni,SP LLC,15:49,NaN


In [7]:
def LateBy(InTime):
    from numpy import nan,NaN,NAN
    officeTime = "09:00"

    if InTime != 0 and type(InTime) != type(0) and InTime != str(0):
        datetimeFormat = '%H:%M'
        intime = datetime.strptime(str(InTime).split('.')[0],datetimeFormat)
        officeLog = datetime.strptime(str(officeTime).split('.')[0],datetimeFormat)
        if intime > officeLog:
            lateby = intime - officeLog
            lateby = lateby.seconds
            hours = lateby//3600 # showing in hour
            minutes = lateby//60
            rest_of_seconds = minutes % 60
            return "%02d:%02d" % (hours,rest_of_seconds)
        else:
            lateby = 0
            return lateby
    else:
        lateby = 0
        return lateby


def EarlyBy(outTime):
    from numpy import nan,NaN,NAN
    offTime = "18:00"

    if outTime != 0 and type(outTime) != type(0) and outTime != str(0):
        datetimeFormat = '%H:%M'
        outtime=datetime.strptime(str(outTime).split('.')[0],datetimeFormat)
        #date2=datetime.strptime(str(offTime).split('.')[0],datetimeFormat)
        offic_time = datetime.strptime(str('18:00').split('.')[0],datetimeFormat)
        if outtime < offic_time:
            Earlyby = offic_time - outtime
            Earlyby = Earlyby.seconds
            hours = Earlyby//3600 # showing in hour
            minutes = Earlyby//60
            rest_of_seconds = minutes % 60
            return "%02d:%02d" % (hours,rest_of_seconds)
        else:
            Earlyby = 0
            return Earlyby
    else:
        Earlyby = 0
        return Earlyby

attendence = pd.DataFrame({"ID":attendence_fnl["Employee_ID"], "Name":attendence_fnl["Employee_Name"],"Department":attendence_fnl["Department"],"DATE":attendence_fnl["DATE"],"IN TIME":attendence_fnl["IN TIME"],"OUT TIME":attendence_fnl["OUT TIME"]})

# Replacing NaN with ZERO's
attendence  = attendence.fillna(value = 0)

# Apply function for IN TIME and OUT TIME
attendence["IN TIME"].apply(LateBy)
attendence["OUT TIME"].apply(EarlyBy)

# Adding new column for LateBY and EarlyBy in Main Data Frame
attendence["LATE BY"] = attendence["IN TIME"].apply(LateBy)
attendence["EarlyBy"] = attendence["OUT TIME"].apply(EarlyBy)
attendence

,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy
123,2258,Kobbana Vasanth Rao,Data Analytics,2021-02-16,17:02,0,08:02,0
68,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2021-02-16,17:01,0,08:01,0
141,6015,Karkarlapudi Vineeth Varma,Data Analytics,2021-02-16,17:04,0,08:04,0
53,2065,Venkata Amaranath Reddy Nandyala,OTC,2020-11-02,15:50,0,06:50,0
51,2056,Sanivarapu Krishnaveni,SP LLC,2020-11-02,15:49,0,06:49,0
109,2225,Batchu Hari Murali Krishna Reddy,OTC,2020-11-02,15:48,0,06:48,0
49,2011,Akhil Varma Uppalapati,ADMS,2020-11-02,15:47,0,06:47,0
47,1996,K Vivekananda Murthy,SP LLC,2020-11-02,15:48,0,06:48,0
111,2226,B Uday Kumar,Data Analytics,2020-11-02,15:48,0,06:48,0
77,2119,Konga Bhavana,Testing,2020-11-02,15:49,0,06:49,0


In [8]:
in_ = []
out_ = []

in_ = [each if each!=0 else 0 for each in attendence["IN TIME"]]
out_ = [each if each!=0 else 0 for each in attendence["OUT TIME"]]
duration1 = []
for v in range(0, len(in_)):

    if in_[v] != 0 and out_[v] != 0:
        if (pd.to_datetime(in_[v])) < (pd.to_datetime(out_[v])):
            diff  =  (pd.to_datetime(out_[v]) - pd.to_datetime(in_[v]))
            diff = str(diff)[-8:-3]
            #print(diff)
            duration1.append(diff)
        else:
            duration1.append(0)
    else:
        duration1.append(0)
len(duration1)
attendence["Total_Duration"] = duration1
attendence

,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy,Total_Duration
123,2258,Kobbana Vasanth Rao,Data Analytics,2021-02-16,17:02,0,08:02,0,0
68,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2021-02-16,17:01,0,08:01,0,0
141,6015,Karkarlapudi Vineeth Varma,Data Analytics,2021-02-16,17:04,0,08:04,0,0
53,2065,Venkata Amaranath Reddy Nandyala,OTC,2020-11-02,15:50,0,06:50,0,0
51,2056,Sanivarapu Krishnaveni,SP LLC,2020-11-02,15:49,0,06:49,0,0
109,2225,Batchu Hari Murali Krishna Reddy,OTC,2020-11-02,15:48,0,06:48,0,0
49,2011,Akhil Varma Uppalapati,ADMS,2020-11-02,15:47,0,06:47,0,0
47,1996,K Vivekananda Murthy,SP LLC,2020-11-02,15:48,0,06:48,0,0
111,2226,B Uday Kumar,Data Analytics,2020-11-02,15:48,0,06:48,0,0
77,2119,Konga Bhavana,Testing,2020-11-02,15:49,0,06:49,0,0


In [9]:
datetimeFormat = '%H:%M'
work_hrs =  datetime.strptime(str("09:00").split('.')[0],datetimeFormat)
#officeLog_in = datetime.strptime(str("09:00").split('.')[0],datetimeFormat)
ot = []
for each in attendence["Total_Duration"]:
    if each != np.nan and each != 0 and each != str(0):
        total_duration = datetime.strptime(str(each).split('.')[0],datetimeFormat)
        if total_duration > work_hrs:
            td = total_duration - work_hrs
            td = td.seconds
            hours = td//3600 # showing in hour
            minutes = td/60
            minute = minutes % 60
            td = "%02d:%02d" % (hours,minute)
            ot.append(td)
        else:
            total_duration = 0
            ot.append(total_duration)
    else:
        total_duration = 0
        ot.append(total_duration)
attendence["OT Minutes"] = ot
attendence

,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy,Total_Duration,OT Minutes
123,2258,Kobbana Vasanth Rao,Data Analytics,2021-02-16,17:02,0,08:02,0,0,0
68,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2021-02-16,17:01,0,08:01,0,0,0
141,6015,Karkarlapudi Vineeth Varma,Data Analytics,2021-02-16,17:04,0,08:04,0,0,0
53,2065,Venkata Amaranath Reddy Nandyala,OTC,2020-11-02,15:50,0,06:50,0,0,0
51,2056,Sanivarapu Krishnaveni,SP LLC,2020-11-02,15:49,0,06:49,0,0,0
109,2225,Batchu Hari Murali Krishna Reddy,OTC,2020-11-02,15:48,0,06:48,0,0,0
49,2011,Akhil Varma Uppalapati,ADMS,2020-11-02,15:47,0,06:47,0,0,0
47,1996,K Vivekananda Murthy,SP LLC,2020-11-02,15:48,0,06:48,0,0,0
111,2226,B Uday Kumar,Data Analytics,2020-11-02,15:48,0,06:48,0,0,0
77,2119,Konga Bhavana,Testing,2020-11-02,15:49,0,06:49,0,0,0


In [10]:
attendence.index = range(0,int(attendence.shape[0]))
duration_e = []
for each in range(0,attendence.shape[0]):
    if attendence["Total_Duration"][each] != 0 and attendence["Total_Duration"][each] != 0 and  attendence["OT Minutes"][each]!= 0:
        ddt = datetime.strptime(attendence["Total_Duration"][each].split('.')[0],datetimeFormat)
        oot = datetime.strptime(attendence["OT Minutes"][each].split('.')[0],datetimeFormat)
        dur = ddt - oot
        dur = dur.seconds
        hours = dur//3600 # showing in hour
        minutes = dur/60
        minute = minutes % 60
        dur = "%02d:%02d" % (hours,minute)
        duration_e.append(dur)
    else:
        dur = attendence["Total_Duration"][each]
        duration_e.append(dur)
attendence["Duration"] = duration_e
attendence

,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy,Total_Duration,OT Minutes,Duration
0,2258,Kobbana Vasanth Rao,Data Analytics,2021-02-16,17:02,0,08:02,0,0,0,0
1,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2021-02-16,17:01,0,08:01,0,0,0,0
2,6015,Karkarlapudi Vineeth Varma,Data Analytics,2021-02-16,17:04,0,08:04,0,0,0,0
3,2065,Venkata Amaranath Reddy Nandyala,OTC,2020-11-02,15:50,0,06:50,0,0,0,0
4,2056,Sanivarapu Krishnaveni,SP LLC,2020-11-02,15:49,0,06:49,0,0,0,0
5,2225,Batchu Hari Murali Krishna Reddy,OTC,2020-11-02,15:48,0,06:48,0,0,0,0
6,2011,Akhil Varma Uppalapati,ADMS,2020-11-02,15:47,0,06:47,0,0,0,0
7,1996,K Vivekananda Murthy,SP LLC,2020-11-02,15:48,0,06:48,0,0,0,0
8,2226,B Uday Kumar,Data Analytics,2020-11-02,15:48,0,06:48,0,0,0,0
9,2119,Konga Bhavana,Testing,2020-11-02,15:49,0,06:49,0,0,0,0


In [11]:
df.shape
df = df.groupby(["DATE","Employee_ID"])["IN TIME"].nunique().reset_index()
df = df.sort_values(by=["DATE","Employee_ID"], axis=0,ascending = False, inplace=False, kind='quicksort', na_position='last')
df.head()

,DATE,Employee_ID,IN TIME
141,2021-02-16,6015,1
140,2021-02-16,2258,1
139,2021-02-16,2108,2
138,2020-11-02,6015,2
137,2020-11-02,6014,1


In [12]:
df1.shape
df1 = df1.groupby(["DATE","Employee_ID"])["OUT TIME"].nunique().reset_index()
df1 = df1.sort_values(by=["DATE","Employee_ID"], axis=0,ascending = False, inplace=False, kind='quicksort', na_position='last')
df1.head()

,DATE,Employee_ID,OUT TIME
19,2020-08-27,2207,1
18,2020-08-27,2108,1
17,2020-08-24,2108,1
16,2020-07-13,2258,1
15,2020-07-13,2207,1


In [13]:
m_df = pd.merge(df,df1, on = ["Employee_ID","DATE"],how = "outer")
m_df.columns = ("DATE","ID","IN_COUNT","OUT_COUNT")
m_df["DATE"] = pd.to_datetime(m_df["DATE"],format =  "%d-%m-%Y")
attendence_n = pd.merge(attendence,m_df, left_on = ["ID","DATE"],right_on =["ID","DATE"], how = "outer" )
attendence_n = attendence_n.fillna(value = 0)
type(attendence_n)
attendence_n["ID"].head(),attendence_n["DATE"].head()

(0    2258
 1    2108
 2    6015
 3    2065
 4    2056
 Name: ID, dtype: int64, 0   2021-02-16
 1   2021-02-16
 2   2021-02-16
 3   2020-11-02
 4   2020-11-02
 Name: DATE, dtype: datetime64[ns])

In [14]:
df_in = pd.read_csv("In Count Data.csv")
# df_in["DATE"] = pd.to_datetime([datetime.strptime(each,"%d-%m-%Y") for each in df_in["DATE"]],format = "%d-%m-%Y")
# df_out = pd.read_sql('SELECT * FROM employee_details.employee_out;',con = database_connection_source)#
df_in["DATE"] = df_in["DATE"].astype('datetime64')
df_out = pd.read_csv("In Count Data2.csv")
# df_out["DATE"] = pd.to_datetime([datetime.strptime(each,"%d-%m-%Y") for each in df_out["DATE"]],format = "%d-%m-%Y")
df_out["DATE"] = df_out["DATE"].astype('datetime64')
df_test = pd.merge(df_in,df_out,on  = ["Employee_ID","Employee_Name","Department","shift_time","DATE","Day"],how='outer')
df_test2 = df_test[["Employee_ID","DATE","IN TIME","OUT TIME"]]
dates = list(df_test2.DATE.unique())
dates
type(df_test2)

pandas.core.frame.DataFrame

In [15]:
data_frame = pd.DataFrame(columns = ["DATE","ID","LOGIN_TIME_ACTIVE"])
for each in dates:
    if each != np.nan:
        d = df_test2[df_test2.DATE == each]
        ids = list(d.Employee_ID.unique())
        for each1 in ids:
            did = d[d.Employee_ID == each1]
            list1 = list(did["IN TIME"].unique())
            list2 = list(did["OUT TIME"].unique())
            list_new = list1 +list2
            if 0 in list_new:
                list_new.remove(0)
            else:
                pass
            list_new = pd.to_datetime(list_new).sort_values()
            in_time = list_new[0::2]
            out_time =list_new[1::2]
            difference = pd.Series(out_time) - pd.Series(in_time)
            added = str(difference.sum()).split(" ")[-1][:-3]
            data_frame_temp = pd.DataFrame({"DATE": each, "ID": each1, "LOGIN_TIME_ACTIVE": added}, index = [0])
            data_frame =  pd.concat([data_frame,data_frame_temp])
    data_frame["ID"] = data_frame["ID"].astype('int64')
    data_frame["DATE"] = data_frame["DATE"].astype('datetime64')
    data_frame["DATE"] = pd.to_datetime(data_frame["DATE"],format="%d-%m-%Y")
# data_frame
data_frame["DATE"].head(),data_frame["ID"].head()
attendence_n1 = pd.merge(attendence_n,data_frame,left_on=["ID","DATE"], right_on=["ID","DATE"])
act_list=[each if each != 0 else 0 for each in attendence_n1["LOGIN_TIME_ACTIVE"]]
attendence_n1["LOGIN_TIME_ACTIVE"] = act_list
# attendence_n1.to_csv("test10.csv",index=False)
attendence_n1

C:\Users\spsoft\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy,Total_Duration,OT Minutes,Duration,IN_COUNT,OUT_COUNT,LOGIN_TIME_ACTIVE
0,2258,Kobbana Vasanth Rao,Data Analytics,2021-02-16,17:02,0,08:02,0,0,0,0,1.0,0.0,00:00
1,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2021-02-16,17:01,0,08:01,0,0,0,0,2.0,0.0,00:01
2,6015,Karkarlapudi Vineeth Varma,Data Analytics,2021-02-16,17:04,0,08:04,0,0,0,0,1.0,0.0,00:00
3,2246,Sanivarapu Taraka Ramalinga Reddy,OTC,2020-08-31,09:39,0,00:39,0,0,0,0,1.0,0.0,00:00
4,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2020-08-31,09:39,0,00:39,0,0,0,0,1.0,0.0,00:00
5,2170,G S Sandeep,OTC,2020-08-31,14:25,0,05:25,0,0,0,0,1.0,0.0,00:00
6,1798,Kovvuru Lokesh,NSG,2020-08-31,14:25,0,05:25,0,0,0,0,1.0,0.0,00:00
7,2258,Kobbana Vasanth Rao,Data Analytics,2020-08-31,09:39,0,00:39,0,0,0,0,2.0,0.0,04:46
8,2137,Kudumula Siva Rama Krishna Reddy,OTC,2020-08-31,12:48,0,03:48,0,0,0,0,1.0,0.0,00:00
9,2207,Polamoni Preethi,Data Analytics,2020-08-31,09:39,0,00:39,0,0,0,0,1.0,0.0,00:00


In [18]:
attendence.shape[0]

154

In [16]:
attendence_n1.index = range(0,int(attendence_n1.shape[0]))
break_time = []
for each in range(0,attendence_n1.shape[0]):
    if attendence_n1["Total_Duration"][each] != 0 and  attendence_n1["LOGIN_TIME_ACTIVE"][each]!= 0:
        total_duration = datetime.strptime(attendence_n1["Total_Duration"][each].split('.')[0],datetimeFormat)
        active_time = datetime.strptime(attendence_n1["LOGIN_TIME_ACTIVE"][each].split('.')[0],datetimeFormat)
        break_ = total_duration - active_time
        break_ = break_.seconds
        hours = break_//3600 # showing in hour
        minutes = break_/60
        minute = minutes % 60
        break_ = "%02d:%02d" % (hours,minute)
        break_time.append(break_)
    else:
        break_ = 0
        break_time.append(break_)
attendence_n1["Break Time"] = break_time
attendence_n1["Break Time"] = attendence_n1["Break Time"].fillna(value = 0)
attendence_n1

,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy,Total_Duration,OT Minutes,Duration,IN_COUNT,OUT_COUNT,LOGIN_TIME_ACTIVE,Break Time
0,2258,Kobbana Vasanth Rao,Data Analytics,2021-02-16,17:02,0,08:02,0,0,0,0,1.0,0.0,00:00,0
1,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2021-02-16,17:01,0,08:01,0,0,0,0,2.0,0.0,00:01,0
2,6015,Karkarlapudi Vineeth Varma,Data Analytics,2021-02-16,17:04,0,08:04,0,0,0,0,1.0,0.0,00:00,0
3,2246,Sanivarapu Taraka Ramalinga Reddy,OTC,2020-08-31,09:39,0,00:39,0,0,0,0,1.0,0.0,00:00,0
4,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2020-08-31,09:39,0,00:39,0,0,0,0,1.0,0.0,00:00,0
5,2170,G S Sandeep,OTC,2020-08-31,14:25,0,05:25,0,0,0,0,1.0,0.0,00:00,0
6,1798,Kovvuru Lokesh,NSG,2020-08-31,14:25,0,05:25,0,0,0,0,1.0,0.0,00:00,0
7,2258,Kobbana Vasanth Rao,Data Analytics,2020-08-31,09:39,0,00:39,0,0,0,0,2.0,0.0,04:46,0
8,2137,Kudumula Siva Rama Krishna Reddy,OTC,2020-08-31,12:48,0,03:48,0,0,0,0,1.0,0.0,00:00,0
9,2207,Polamoni Preethi,Data Analytics,2020-08-31,09:39,0,00:39,0,0,0,0,1.0,0.0,00:00,0


In [17]:
status = []
a = np.nan
for each in range(0,attendence_n1.shape[0]):
    if (attendence_n1["IN TIME"][each] != 0 or attendence_n1["OUT TIME"][each] != 0) and type(attendence_n1["IN TIME"][each]) != type(a)  and type(attendence_n1["OUT TIME"][each]) != type(a):
        status.append("Present")
    else:
        status.append("unknown")

attendence_n1["status"] = status
# attendence_n1.to_csv("test5.csv",index = False)
# df["DATE"]
attendence_n1

,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy,Total_Duration,OT Minutes,Duration,IN_COUNT,OUT_COUNT,LOGIN_TIME_ACTIVE,Break Time,status
0,2258,Kobbana Vasanth Rao,Data Analytics,2021-02-16,17:02,0,08:02,0,0,0,0,1.0,0.0,00:00,0,Present
1,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2021-02-16,17:01,0,08:01,0,0,0,0,2.0,0.0,00:01,0,Present
2,6015,Karkarlapudi Vineeth Varma,Data Analytics,2021-02-16,17:04,0,08:04,0,0,0,0,1.0,0.0,00:00,0,Present
3,2246,Sanivarapu Taraka Ramalinga Reddy,OTC,2020-08-31,09:39,0,00:39,0,0,0,0,1.0,0.0,00:00,0,Present
4,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,2020-08-31,09:39,0,00:39,0,0,0,0,1.0,0.0,00:00,0,Present
5,2170,G S Sandeep,OTC,2020-08-31,14:25,0,05:25,0,0,0,0,1.0,0.0,00:00,0,Present
6,1798,Kovvuru Lokesh,NSG,2020-08-31,14:25,0,05:25,0,0,0,0,1.0,0.0,00:00,0,Present
7,2258,Kobbana Vasanth Rao,Data Analytics,2020-08-31,09:39,0,00:39,0,0,0,0,2.0,0.0,04:46,0,Present
8,2137,Kudumula Siva Rama Krishna Reddy,OTC,2020-08-31,12:48,0,03:48,0,0,0,0,1.0,0.0,00:00,0,Present
9,2207,Polamoni Preethi,Data Analytics,2020-08-31,09:39,0,00:39,0,0,0,0,1.0,0.0,00:00,0,Present


# Final Attendance

In [18]:
Emp_df =  pd.read_csv('dataOFEmp.csv')
Emp_DF = pd.DataFrame({"ID": Emp_df["Employee_ID"],"Name":Emp_df["Employee_Name"],"Department":Emp_df["Department"]})
attendence_final = pd.DataFrame(columns = ["ID","Name","Department","DATE", "IN TIME","OUT TIME", "LATE BY","EarlyBy", "Total_Duration","OT Minutes", "Duration","IN_COUNT","OUT_COUNT","LOGIN_TIME_ACTIVE","Break Time","status"])
attendence_final

,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy,Total_Duration,OT Minutes,Duration,IN_COUNT,OUT_COUNT,LOGIN_TIME_ACTIVE,Break Time,status


In [19]:
for each in attendence_n1["DATE"].unique():
    time= pd.to_datetime(each)
    #time  = time.strftime("%d-%m-%Y")
    date_wise = attendence_n1[attendence_n1["DATE"] == time]
    #print(date_wise,"\n")

    final_attendence = pd.merge(Emp_DF,date_wise, on =["ID","Name","Department"],how='outer')
    final_attendence = final_attendence.fillna(value = 0)
#final_attendence["DATE"].unique()
    #final_attendence["DATE"] = [time.strftime("%d-%m-%Y") if type(each) == type(f) else each.strftime("%d-%m-%Y") for each in final_attendence["DATE"]]


    final_attendence["DATE"] = [time.strftime("%d-%m-%Y") if each == 0 else each.strftime("%d-%m-%Y") for each in final_attendence["DATE"]]
    final_attendence["status"] =  ["Absent" if each == 0 else each for each in final_attendence["status"]]
    final_attendence = final_attendence.sort_values(by = ["DATE","IN TIME","status"], ascending=False)
    attendence_final  = pd.concat([final_attendence,attendence_final], ignore_index = True)
attendence_final

,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy,Total_Duration,OT Minutes,Duration,IN_COUNT,OUT_COUNT,LOGIN_TIME_ACTIVE,Break Time,status
0,2207,Polamoni Preethi,Data Analytics,18-06-2020,16:52,16:54,07:52,01:06,00:02,0.0,00:02,1.0,1.0,00:02,00:00,Present
1,1879,Are Venkateswara Reddy,NSG,18-06-2020,11:32,16:53,02:32,01:07,05:21,0.0,05:21,4.0,2.0,01:21,04:00,Present
2,2108,Parisa Bala Venkata Sai Raghava,Data Analytics,18-06-2020,11:31,16:53,02:31,01:07,05:22,0.0,05:22,7.0,3.0,05:18,00:04,Present
3,2258,Kobbana Vasanth Rao,Data Analytics,18-06-2020,11:29,16:53,02:29,01:07,05:24,0.0,05:24,7.0,3.0,05:20,00:04,Present
4,2139,Shaik Ibraheem Khaleelulla,Testing,18-06-2020,11:29,0,02:29,0,0,0.0,0,1.0,0.0,00:00,0,Present
5,1798,Kovvuru Lokesh,NSG,18-06-2020,0,16:53,0,01:07,0,0.0,0,0.0,1.0,00:00,0,Present
6,6007,Abhishek Kumar,Intern,18-06-2020,0,12:52,0,05:08,0,0.0,0,0.0,1.0,00:00,0,Present
7,1,Ch. Sreeman Kumar,Corp,18-06-2020,0,0,0,0,0,0.0,0,0.0,0.0,0,0,Absent
8,9,Pulla Reddy Sir,MD,18-06-2020,0,0,0,0,0,0.0,0,0.0,0.0,0,0,Absent
9,13,Satish Karri,ADMS,18-06-2020,0,0,0,0,0,0.0,0,0.0,0.0,0,0,Absent


In [20]:
present_day_attdence = attendence_final[attendence_final.DATE == datetime.now().strftime("%d-%m-%Y")]

present_day_attdence[["IN_COUNT","OUT_COUNT"]] = present_day_attdence[["IN_COUNT","OUT_COUNT"]].astype(int)

li = list(attendence_final["DATE"].unique())
date_now = datetime.now().strftime("%d-%m-%Y")
date_now

if date_now not in li:
    present_day_attdence1 = pd.DataFrame(columns = ["ID","Name","Department","DATE", "IN TIME","OUT TIME", "LATE BY","EarlyBy", "Total_Duration","OT Minutes", "Duration","IN_COUNT","OUT_COUNT","LOGIN_TIME_ACTIVE","Break Time","status"])
    present_day_attdence1["ID"],present_day_attdence1["Name"],present_day_attdence1["Department"]=Emp_df["Employee_ID"],Emp_df["Employee_Name"],Emp_df["Department"]
    present_day_attdence1["DATE"] = [date_now for each in range(len(Emp_df))]
    present_day_attdence1["IN TIME"],present_day_attdence1["OUT TIME"],present_day_attdence1["LATE BY"]=[0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))]
    present_day_attdence1["EarlyBy"],present_day_attdence1["Total_Duration"],present_day_attdence1["OT Minutes"]= [0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))]
    present_day_attdence1["Duration"],present_day_attdence1["IN_COUNT"],present_day_attdence1["OUT_COUNT"] = [0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))]
    present_day_attdence1["LOGIN_TIME_ACTIVE"],present_day_attdence1["Break Time"] = [0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))]
    present_day_attdence1["status"] =["Absent" for each in range(len(Emp_df))]
    
    attendence_final = pd.concat([attendence_final,present_day_attdence1],ignore_index = True)

else:
    present_day_attdence1 = present_day_attdence

attendence_final=attendence_final.sort_values(by=["DATE","status"], axis=0,ascending = True, inplace=False, kind='quicksort', na_position='last')
# attendence_final.to_csv("overall_attendence_final.csv",index = False)
# present_day_attdence1.to_csv("present_day_attdence.csv",index = False)
present_day_attdence1

,ID,Name,Department,DATE,IN TIME,OUT TIME,LATE BY,EarlyBy,Total_Duration,OT Minutes,Duration,IN_COUNT,OUT_COUNT,LOGIN_TIME_ACTIVE,Break Time,status
0,1,Ch. Sreeman Kumar,Corp,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent
1,9,Pulla Reddy Sir,MD,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent
2,13,Satish Karri,ADMS,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent
3,24,Arindam Chakraborty,TAG,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent
4,29,T. P. Sukeshini,Corp,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent
5,42,K.Satyanarayana Murty,Corp,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent
6,265,D.Sanjeev Kumar,SP LLC,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent
7,281,R.Kishore Kumar,Multimedia,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent
8,336,K.V.Sudhakara Reddy,Citadel,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent
9,483,K.Sreedhar,Citadel,22-02-2021,0,0,0,0,0,0,0,0,0,0,0,Absent


In [24]:
# attendence_final,present_day_attdence1 = manuplations_attendance()

def get_time_diff_in_seconds(present_time,last_updated_time):
    datetimeFormat = '%Y-%m-%d %H:%M:%S'
    date1=datetime.strptime(str(present_time).split('.')[0],datetimeFormat)
    date2=datetime.strptime(str(last_updated_time).split('.')[0],datetimeFormat)
    diff = date2-date1
    diff=diff.seconds
    return diff

def overall_subject():
    attendence_final["DATE"]=attendence_final["DATE"].astype('datetime64')
    min_time = datetime.strftime(min(attendence_final["DATE"]),'%d-%m-%Y')
    Overall_subject_mail = "Employee Timesheet From " + min_time +' - ' + str(datetime.now().strftime("%d-%m-%Y"))
    return Overall_subject_mail

def present_subject():
    Present_subject_mail = "Employee Timesheet On " + str(datetime.now().strftime("%d-%m-%Y"))
    return Present_subject_mail

def overall_body():
    attendence_final["DATE"]=attendence_final["DATE"].astype('datetime64')
    min_time = datetime.strftime(min(attendence_final["DATE"]),'%d-%m-%Y')
    Overall_body_mail = 'Dear Sir,'+'\n'+'\n'+'\n'+'Please find the attachment of Employee Timesheet From '+min_time+' - '+str(datetime.now().strftime("%d-%m-%Y"))+'.\n\n\n'+'Regards,'+'\n'+'ADMS-Data Analytics'+'\n'+'\n'+'\n'+'This is an System Generated E-mail. If you have any queries Please contact to the E-mail id: sairaghava.parisa@spsoft.in'
    return Overall_body_mail

def present_body():
    Present_body_mail = 'Dear Sir,'+'\n'+'\n'+'\n'+'Please find the attachment of Employee Timesheet On '+str(datetime.now().strftime("%d-%m-%Y"))+'.\n\n\n'+'Regards,'+'\n'+'ADMS-Data Analytics'+'\n'+'\n'+'\n'+'This is an System Generated E-mail. If you have any queries Please contact to the E-mail id: sairaghava.parisa@spsoft.in'
    return Present_body_mail

start_time =pd.to_datetime(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))

from_address="vineeth.kakarlapudi@spsoft.in"
names_send_dict={"ADMS-Data Analytics":"preethi.polamoni@spsoft.in","Java":"sairaghava.parisa@spsoft.in"}

def get_mail(name):
    return names_send_dict.get(name)

#now_true = datetime.now().strftime('%H:%M:%S')
#send_mail=parameter
while True:
    present_time= pd.to_datetime(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
    counter_time_in_seconds=get_time_diff_in_seconds(start_time,present_time)
    email_send_time_counter_hours=counter_time_in_seconds/60

    #     if datetime.now().strftime('%H:%M:%S')==send_mail:
    if (email_send_time_counter_hours==5):
#         manuplations_attendance()
        time.sleep(1)
        print('sending_mail')
        
        overall_attendence=pd.read_csv('overall_attendence_final_new_break.csv')
        present_day=pd.read_csv('present_day_attdence.csv')

        username = from_address
        password = 'spsoft@123'


        for key in names_send_dict.keys():
            #filter_by_name=overall_attendence[overall_attendence['Department']==str(key)]
            #filter_by_name.to_csv("overall_attendence_dept.csv")
            msg = MIMEMultipart()
            msg["From"] = from_address
            to_address =get_mail(key)
            print("Overall",to_address)
            msg["To"] = to_address
            msg["Subject"] = overall_subject()

            body = overall_body()
            body = MIMEText(body) # convert the body to a MIME compatible string
            msg.attach(body)

            attachment_path_1='overall_attendence_final_new_break.csv'
            attachment_1 = MIMEBase('application', "octet-stream")
            attachment_1.set_payload(open(attachment_path_1, "rb").read())
            encoders.encode_base64(attachment_1)
            attachment_1.add_header('Content-Disposition', 'attachment', filename= str(datetime.now().strftime("%d-%m-%Y"))+' overall_attendence.csv') 
            msg.attach(attachment_1)

            server = smtplib.SMTP('sg3plvcpnl208309.prod.sin3.secureserver.net')
            server.starttls()
            server.login(username,password)
            server.sendmail(from_address,to_address,msg.as_string())
            server.quit()


        for key in names_send_dict.keys():
            #filter_by_name1=present_day[present_day['Department']==str(key)]
            #filter_by_name1.to_csv("present_day_dept.csv")
            msg = MIMEMultipart()
            msg["From"] = from_address
            to_address =get_mail(key)
            print("Present Day",to_address)
            msg["To"] = to_address
            msg["Subject"] = present_subject()

            body = present_body()
            body = MIMEText(body) # convert the body to a MIME compatible string
            msg.attach(body)

            attachment_path_2='present_day_attdence.csv'
            attachment_2 = MIMEBase('application', "octet-stream")
            attachment_2.set_payload(open(attachment_path_2, "rb").read())
            encoders.encode_base64(attachment_2)
            attachment_2.add_header('Content-Disposition', 'attachment', filename= str(datetime.now().strftime("%d-%m-%Y"))+' present_day_attdence.csv') 
            msg.attach(attachment_2)

            server = smtplib.SMTP('sg3plvcpnl208309.prod.sin3.secureserver.net')
            server.starttls()
            server.login(username,password)
            server.sendmail(from_address,to_address,msg.as_string())
            server.quit()
        start_time=present_time

sending_mail
Overall preethi.polamoni@spsoft.in
Overall sairaghava.parisa@spsoft.in
Present Day preethi.polamoni@spsoft.in
Present Day sairaghava.parisa@spsoft.in
sending_mail
Overall preethi.polamoni@spsoft.in
Overall sairaghava.parisa@spsoft.in
Present Day preethi.polamoni@spsoft.in
Present Day sairaghava.parisa@spsoft.in
sending_mail
Overall preethi.polamoni@spsoft.in
Overall sairaghava.parisa@spsoft.in
Present Day preethi.polamoni@spsoft.in
Present Day sairaghava.parisa@spsoft.in


KeyboardInterrupt: 

In [35]:
time.sleep(10)
print("hi")

hi


In [9]:
def get_time_diff_in_seconds(present_time,last_updated_time):
    datetimeFormat = '%Y-%m-%d %H:%M:%S'
    date1=datetime.strptime(str(present_time).split('.')[0],datetimeFormat)
    date2=datetime.strptime(str(last_updated_time).split('.')[0],datetimeFormat)
    diff = date2-date1
    diff=diff.seconds
    return diff


In [22]:
import time

In [8]:
present_time= pd.to_datetime(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
start_time =pd.to_datetime(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
get_time_diff_in_seconds(start_time,present_time)

0

In [ ]:
present_time= pd.to_datetime(datetime.today().strftime('%H:%M:%S'))
present_time

In [2]:
while True:
    print("hi" + str(datetime.now().strftime("%d-%m-%Y %H:%M:%S")))
    time.sleep(60)


hi23-02-2021 11:31:21
hi23-02-2021 11:32:21


KeyboardInterrupt: 

In [6]:
def hello():
    print("hi /n"+str(datetime.now().strftime("%d-%m-%Y %H:%M:%S:")) )
while True:
    hello()
    time.sleep(60)
# hello()

hi /n23-02-2021 11:42:07:
hi /n23-02-2021 11:43:07:


KeyboardInterrupt: 

# WORKING CODE

In [ ]:
import pandas as pd
import time
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from datetime import datetime
from datetime import datetime,timedelta
import mysql.connector
import pandas as pd
import numpy as np
import sqlalchemy
import pymysql
import mysql
from flask import Flask

# def manipulations():
def manuplations_attendance():
    df = pd.read_csv('In Count Data.csv')
    # df1["DATE"].head()

    df["DATE"] = df["DATE"].astype('datetime64')
    df["DATE"].head()
    a = df[["Employee_ID","Employee_Name","Department","DATE","IN TIME"]]
    dd = df.groupby(["Employee_ID","DATE"])["Employee_Name","Department","IN TIME"].min()
    in_time = dd.reset_index()
    # in_time.head()
    df1 = pd.read_csv('In Count Data2.csv')
    df1["DATE"]= df1["DATE"].astype('datetime64')
    df1["DATE"].head()
    # Out Time min()
    dfff = df1[["Employee_ID","Employee_Name","Department","DATE","OUT TIME"]]
    ddd = dfff.groupby(["Employee_ID","DATE"])["Employee_Name","Department","OUT TIME"].max()
    out_time = ddd.reset_index()
    # out_time["DATE"].head()
    atn = pd.merge(in_time, out_time, on = ["Employee_ID","Employee_Name","Department","DATE"], how = "outer")
    attendence_fnl = atn.sort_values(by=["DATE"], axis=0,ascending = False, inplace=False, kind='quicksort', na_position='last')
    td = attendence_fnl[attendence_fnl["DATE"] == datetime.now().strftime("%d-%b-%Y")]
    # td
    # attendence_fnl.head()
    def LateBy(InTime):
        from numpy import nan,NaN,NAN
        officeTime = "09:00"

        if InTime != 0 and type(InTime) != type(0) and InTime != str(0):
            datetimeFormat = '%H:%M'
            intime = datetime.strptime(str(InTime).split('.')[0],datetimeFormat)
            officeLog = datetime.strptime(str(officeTime).split('.')[0],datetimeFormat)
            if intime > officeLog:
                lateby = intime - officeLog
                lateby = lateby.seconds
                hours = lateby//3600 # showing in hour
                minutes = lateby//60
                rest_of_seconds = minutes % 60
                return "%02d:%02d" % (hours,rest_of_seconds)
            else:
                lateby = 0
                return lateby
        else:
            lateby = 0
            return lateby


    def EarlyBy(outTime):
        from numpy import nan,NaN,NAN
        offTime = "18:00"

        if outTime != 0 and type(outTime) != type(0) and outTime != str(0):
            datetimeFormat = '%H:%M'
            outtime=datetime.strptime(str(outTime).split('.')[0],datetimeFormat)
            #date2=datetime.strptime(str(offTime).split('.')[0],datetimeFormat)
            offic_time = datetime.strptime(str('18:00').split('.')[0],datetimeFormat)
            if outtime < offic_time:
                Earlyby = offic_time - outtime
                Earlyby = Earlyby.seconds
                hours = Earlyby//3600 # showing in hour
                minutes = Earlyby//60
                rest_of_seconds = minutes % 60
                return "%02d:%02d" % (hours,rest_of_seconds)
            else:
                Earlyby = 0
                return Earlyby
        else:
            Earlyby = 0
            return Earlyby

    attendence = pd.DataFrame({"ID":attendence_fnl["Employee_ID"], "Name":attendence_fnl["Employee_Name"],"Department":attendence_fnl["Department"],"DATE":attendence_fnl["DATE"],"IN TIME":attendence_fnl["IN TIME"],"OUT TIME":attendence_fnl["OUT TIME"]})

    # Replacing NaN with ZERO's
    attendence  = attendence.fillna(value = 0)

    # Apply function for IN TIME and OUT TIME
    attendence["IN TIME"].apply(LateBy)
    attendence["OUT TIME"].apply(EarlyBy)

    # Adding new column for LateBY and EarlyBy in Main Data Frame
    attendence["LATE BY"] = attendence["IN TIME"].apply(LateBy)
    attendence["EarlyBy"] = attendence["OUT TIME"].apply(EarlyBy)
    attendence
    in_ = []
    out_ = []

    in_ = [each if each!=0 else 0 for each in attendence["IN TIME"]]
    out_ = [each if each!=0 else 0 for each in attendence["OUT TIME"]]
    duration1 = []
    for v in range(0, len(in_)):

        if in_[v] != 0 and out_[v] != 0:
            if (pd.to_datetime(in_[v])) < (pd.to_datetime(out_[v])):
                diff  =  (pd.to_datetime(out_[v]) - pd.to_datetime(in_[v]))
                diff = str(diff)[-8:-3]
                #print(diff)
                duration1.append(diff)
            else:
                duration1.append(0)
        else:
            duration1.append(0)
    len(duration1)
    attendence["Total_Duration"] = duration1
    attendence
    datetimeFormat = '%H:%M'
    work_hrs =  datetime.strptime(str("09:00").split('.')[0],datetimeFormat)
    #officeLog_in = datetime.strptime(str("09:00").split('.')[0],datetimeFormat)
    ot = []
    for each in attendence["Total_Duration"]:
        if each != np.nan and each != 0 and each != str(0):
            total_duration = datetime.strptime(str(each).split('.')[0],datetimeFormat)
            if total_duration > work_hrs:
                td = total_duration - work_hrs
                td = td.seconds
                hours = td//3600 # showing in hour
                minutes = td/60
                minute = minutes % 60
                td = "%02d:%02d" % (hours,minute)
                ot.append(td)
            else:
                total_duration = 0
                ot.append(total_duration)
        else:
            total_duration = 0
            ot.append(total_duration)
    attendence["OT Minutes"] = ot
    attendence
    attendence.index = range(0,int(attendence.shape[0]))
    duration_e = []
    for each in range(0,attendence.shape[0]):
        if attendence["Total_Duration"][each] != 0 and attendence["Total_Duration"][each] != 0 and  attendence["OT Minutes"][each]!= 0:
            ddt = datetime.strptime(attendence["Total_Duration"][each].split('.')[0],datetimeFormat)
            oot = datetime.strptime(attendence["OT Minutes"][each].split('.')[0],datetimeFormat)
            dur = ddt - oot
            dur = dur.seconds
            hours = dur//3600 # showing in hour
            minutes = dur/60
            minute = minutes % 60
            dur = "%02d:%02d" % (hours,minute)
            duration_e.append(dur)
        else:
            dur = attendence["Total_Duration"][each]
            duration_e.append(dur)
    attendence["Duration"] = duration_e
    attendence
    df.shape
    df = df.groupby(["DATE","Employee_ID"])["IN TIME"].nunique().reset_index()
    df = df.sort_values(by=["DATE","Employee_ID"], axis=0,ascending = False, inplace=False, kind='quicksort', na_position='last')
    # df.head()
    df1.shape
    df1 = df1.groupby(["DATE","Employee_ID"])["OUT TIME"].nunique().reset_index()
    df1 = df1.sort_values(by=["DATE","Employee_ID"], axis=0,ascending = False, inplace=False, kind='quicksort', na_position='last')
    # df1.head()
    m_df = pd.merge(df,df1, on = ["Employee_ID","DATE"],how = "outer")
    m_df.columns = ("DATE","ID","IN_COUNT","OUT_COUNT")
    m_df["DATE"] = pd.to_datetime(m_df["DATE"],format =  "%d-%m-%Y")
    attendence_n = pd.merge(attendence,m_df, left_on = ["ID","DATE"],right_on =["ID","DATE"], how = "outer" )
    attendence_n = attendence_n.fillna(value = 0)
    attendence_n
    # attendence_n["ID"].head(),attendence_n["DATE"].head()
    df_in = pd.read_csv("In Count Data.csv")
    # df_in["DATE"] = pd.to_datetime([datetime.strptime(each,"%d-%m-%Y") for each in df_in["DATE"]],format = "%d-%m-%Y")
    # df_out = pd.read_sql('SELECT * FROM employee_details.employee_out;',con = database_connection_source)#
    df_in["DATE"] = df_in["DATE"].astype('datetime64')
    df_out = pd.read_csv("In Count Data2.csv")
    # df_out["DATE"] = pd.to_datetime([datetime.strptime(each,"%d-%m-%Y") for each in df_out["DATE"]],format = "%d-%m-%Y")
    df_out["DATE"] = df_out["DATE"].astype('datetime64')
    df_test = pd.merge(df_in,df_out,on  = ["Employee_ID","Employee_Name","Department","shift_time","DATE","Day"],how='outer')
    df_test2 = df_test[["Employee_ID","DATE","IN TIME","OUT TIME"]]
    dates = list(df_test2.DATE.unique())
    dates
    # type(df_test2)\
    data_frame = pd.DataFrame(columns = ["DATE","ID","LOGIN_TIME_ACTIVE"])
    for each in dates:
        if each != np.nan:
            d = df_test2[df_test2.DATE == each]
            ids = list(d.Employee_ID.unique())
            for each1 in ids:
                did = d[d.Employee_ID == each1]
                list1 = list(did["IN TIME"].unique())
                list2 = list(did["OUT TIME"].unique())
                list_new = list1 +list2
                if 0 in list_new:
                    list_new.remove(0)
                else:
                    pass
                list_new = pd.to_datetime(list_new).sort_values()
                in_time = list_new[0::2]
                out_time =list_new[1::2]
                difference = pd.Series(out_time) - pd.Series(in_time)
                added = str(difference.sum()).split(" ")[-1][:-3]
                data_frame_temp = pd.DataFrame({"DATE": each, "ID": each1, "LOGIN_TIME_ACTIVE": added}, index = [0])
                data_frame =  pd.concat([data_frame,data_frame_temp])
        data_frame["ID"] = data_frame["ID"].astype('int64')
        data_frame["DATE"] = data_frame["DATE"].astype('datetime64')
        data_frame["DATE"] = pd.to_datetime(data_frame["DATE"],format="%d-%b-%Y")
    # data_frame
    data_frame["DATE"].head(),data_frame["ID"].head()
    attendence_n1 = pd.merge(attendence_n,data_frame,left_on=["ID","DATE"], right_on=["ID","DATE"])
    act_list=[each if each != 0 else 0 for each in attendence_n1["LOGIN_TIME_ACTIVE"]]
    attendence_n1["LOGIN_TIME_ACTIVE"] = act_list
    # attendence_n1.to_csv("test10.csv",index=False)
    attendence_n1
    attendence_n1.index = range(0,int(attendence_n1.shape[0]))
    break_time = []
    for each in range(0,attendence_n1.shape[0]):
        if attendence_n1["Total_Duration"][each] != 0 and  attendence_n1["LOGIN_TIME_ACTIVE"][each]!= 0:
            total_duration = datetime.strptime(attendence_n1["Total_Duration"][each].split('.')[0],datetimeFormat)
            active_time = datetime.strptime(attendence_n1["LOGIN_TIME_ACTIVE"][each].split('.')[0],datetimeFormat)
            break_ = total_duration - active_time
            break_ = break_.seconds
            hours = break_//3600 # showing in hour
            minutes = break_/60
            minute = minutes % 60
            break_ = "%02d:%02d" % (hours,minute)
            break_time.append(break_)
        else:
            break_ = 0
            break_time.append(break_)
    attendence_n1["Break Time"] = break_time
    attendence_n1["Break Time"] = attendence_n1["Break Time"].fillna(value = 0)
    attendence_n1
    status = []
    a = np.nan
    for each in range(0,attendence_n1.shape[0]):
        if (attendence_n1["IN TIME"][each] != 0 or attendence_n1["OUT TIME"][each] != 0) and type(attendence_n1["IN TIME"][each]) != type(a)  and type(attendence_n1["OUT TIME"][each]) != type(a):
            status.append("Present")
        else:
            status.append("unknown")

    attendence_n1["status"] = status
    # attendence_n1.to_csv("test5.csv",index = False)
    # df["DATE"]
    attendence_n1
    Emp_df =  pd.read_csv('dataOFEmp.csv')
    Emp_DF = pd.DataFrame({"ID": Emp_df["Employee_ID"],"Name":Emp_df["Employee_Name"],"Department":Emp_df["Department"]})
    attendence_final = pd.DataFrame(columns = ["ID","Name","Department","DATE", "IN TIME","OUT TIME", "LATE BY","EarlyBy", "Total_Duration","OT Minutes", "Duration","IN_COUNT","OUT_COUNT","LOGIN_TIME_ACTIVE","Break Time","status"])
    attendence_final
    for each in attendence_n1["DATE"].unique():
        time= pd.to_datetime(each)
        #time  = time.strftime("%d-%m-%Y")
        date_wise = attendence_n1[attendence_n1["DATE"] == time]
        #print(date_wise,"\n")

        final_attendence = pd.merge(Emp_DF,date_wise, on =["ID","Name","Department"],how='outer')
        final_attendence = final_attendence.fillna(value = 0)
    #final_attendence["DATE"].unique()
        #final_attendence["DATE"] = [time.strftime("%d-%m-%Y") if type(each) == type(f) else each.strftime("%d-%m-%Y") for each in final_attendence["DATE"]]


        final_attendence["DATE"] = [time.strftime("%d-%b-%Y") if each == 0 else each.strftime("%d-%b-%Y") for each in final_attendence["DATE"]]
        final_attendence["status"] =  ["Absent" if each == 0 else each for each in final_attendence["status"]]
        final_attendence = final_attendence.sort_values(by = ["DATE","IN TIME","status"], ascending=False)
        attendence_final  = pd.concat([final_attendence,attendence_final], ignore_index = True)
    attendence_final
    present_day_attdence = attendence_final[attendence_final.DATE == datetime.now().strftime("%d-%b-%Y")]

    present_day_attdence[["IN_COUNT","OUT_COUNT"]] = present_day_attdence[["IN_COUNT","OUT_COUNT"]].astype(int)

    li = list(attendence_final["DATE"].unique())
    date_now = datetime.now().strftime("%d-%b-%Y")
    date_now

    if date_now not in li:
        present_day_attdence1 = pd.DataFrame(columns = ["ID","Name","Department","DATE", "IN TIME","OUT TIME", "LATE BY","EarlyBy", "Total_Duration","OT Minutes", "Duration","IN_COUNT","OUT_COUNT","LOGIN_TIME_ACTIVE","Break Time","status"])
        present_day_attdence1["ID"],present_day_attdence1["Name"],present_day_attdence1["Department"]=Emp_df["Employee_ID"],Emp_df["Employee_Name"],Emp_df["Department"]
        present_day_attdence1["DATE"] = [date_now for each in range(len(Emp_df))]
        present_day_attdence1["IN TIME"],present_day_attdence1["OUT TIME"],present_day_attdence1["LATE BY"]=[0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))]
        present_day_attdence1["EarlyBy"],present_day_attdence1["Total_Duration"],present_day_attdence1["OT Minutes"]= [0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))]
        present_day_attdence1["Duration"],present_day_attdence1["IN_COUNT"],present_day_attdence1["OUT_COUNT"] = [0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))]
        present_day_attdence1["LOGIN_TIME_ACTIVE"],present_day_attdence1["Break Time"] = [0 for each in range(len(Emp_df))],[0 for each in range(len(Emp_df))]
        present_day_attdence1["status"] =["Absent" for each in range(len(Emp_df))]

        attendence_final = pd.concat([attendence_final,present_day_attdence1],ignore_index = True)

    else:
        present_day_attdence1 = present_day_attdence

    attendence_final["DATE"]=attendence_final["DATE"].astype('datetime64')
    attendence_final=attendence_final.sort_values(by=["DATE","status"], axis=0,ascending = False, inplace=False)
    attendence_final['DATE'] = attendence_final['DATE'].dt.strftime('%d-%b-%Y')
    attendence_final.to_csv("overall_attendence_final.csv",index = False)
    present_day_attdence1.to_csv("present_day_attdence1.csv",index = False)
    present_day_attdence1
while True:
    manuplations_attendance()
    print("hi " + str(datetime.now().strftime("%d-%m-%Y %H:%M:%S")))
    time.sleep(300)

        
# app = Flask(__name__)
# @app.route('/manipulations', methods = ['GET'])
# def test_run():
#     manipulations()
#     k='success'
#     return k

# if __name__ == '__main__':
#     app.run()

C:\Users\spsoft\Anaconda3\lib\site-packages\ipykernel_launcher.py:196: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


hi 01-03-2021 10:01:55


C:\Users\spsoft\Anaconda3\lib\site-packages\pandas\core\frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


hi 01-03-2021 10:06:56
hi 01-03-2021 10:11:58
hi 01-03-2021 10:16:59
hi 01-03-2021 10:22:01
hi 01-03-2021 10:27:03
hi 01-03-2021 10:32:03
hi 01-03-2021 10:37:04
hi 01-03-2021 10:42:04
hi 01-03-2021 10:47:05
hi 01-03-2021 10:52:06
hi 01-03-2021 10:57:06
hi 01-03-2021 11:02:07
hi 01-03-2021 11:07:08
hi 01-03-2021 11:12:08
hi 01-03-2021 11:17:09
hi 01-03-2021 11:22:10
hi 01-03-2021 11:27:10
hi 01-03-2021 11:32:11
hi 01-03-2021 11:37:12
hi 01-03-2021 11:42:13
hi 01-03-2021 11:47:14
hi 01-03-2021 11:52:15
hi 01-03-2021 11:57:15
hi 01-03-2021 12:02:16
hi 01-03-2021 12:07:17
hi 01-03-2021 12:12:17
hi 01-03-2021 12:17:18
hi 01-03-2021 12:22:19
hi 01-03-2021 12:27:19
hi 01-03-2021 12:32:20
hi 01-03-2021 12:37:21
hi 01-03-2021 12:42:21
hi 01-03-2021 12:47:22
hi 01-03-2021 12:52:22
hi 01-03-2021 12:57:23
hi 01-03-2021 13:02:24
hi 01-03-2021 13:07:24
hi 01-03-2021 13:12:25
hi 01-03-2021 13:17:26
hi 01-03-2021 13:22:26
hi 01-03-2021 13:27:27
hi 01-03-2021 13:32:27
hi 01-03-2021 13:37:28
hi 01-03-20

In [ ]:
#     return type(attendence_final["DATE"])
# #     attendence_final['DATE'] = attendence_final['DATE'].dt.strftime('%d-%b-%Y')
#     attendence_final.to_csv("overall_attendence_final.csv",index = False)
#     present_day_attdence1.to_csv("present_day_attdence.csv",index = False)
#     present_day_attdence1
# while True:
# manuplations_attendance()
#     print("hi " + str(datetime.now().strftime("%d-%m-%Y %H:%M:%S")))
#     time.sleep(30)

In [16]:
# attendence_final,present_day_attdence1 = manuplations_attendance()
attendence_final = pd.read_csv('overall_attendence_final.csv')
present_day_attdence1 = pd.read_csv("present_day_attdence.csv")

def get_time_diff_in_seconds(present_time,last_updated_time):
    datetimeFormat = '%Y-%m-%d %H:%M:%S'
    date1=datetime.strptime(str(present_time).split('.')[0],datetimeFormat)
    date2=datetime.strptime(str(last_updated_time).split('.')[0],datetimeFormat)
    diff = date2-date1
    diff=diff.seconds
    return diff

################################################## email Subject  #######################################################
def overall_subject():
    attendence_final["DATE"]=attendence_final["DATE"].astype('datetime64')
    min_time = datetime.strftime(min(attendence_final["DATE"]),'%d-%m-%Y')
    Overall_subject_mail = "Employee Timesheet From " + min_time +' - ' + str(datetime.now().strftime("%d-%m-%Y"))
    return Overall_subject_mail

def present_subject():
    Present_subject_mail = "Employee Timesheet On " + str(datetime.now().strftime("%d-%m-%Y"))
    return Present_subject_mail

################################################## email body  ##########################################################
def overall_body():
    attendence_final["DATE"]=attendence_final["DATE"].astype('datetime64')
    min_time = datetime.strftime(min(attendence_final["DATE"]),'%d-%m-%Y')
    Overall_body_mail = 'Dear Sir,'+'\n'+'\n'+'\n'+'Please find the attachment of Employee Timesheet From '+min_time+' - '+str(datetime.now().strftime("%d-%m-%Y"))+'.\n\n\n'+'Regards,'+'\n'+'ADMS-Data Analytics'+'\n'+'\n'+'\n'+'This is an System Generated E-mail. If you have any queries Please contact to the E-mail id: sairaghava.parisa@spsoft.in'
    return Overall_body_mail

def present_body():
    Present_body_mail = 'Dear Sir,'+'\n'+'\n'+'\n'+'Please find the attachment of Employee Timesheet On '+str(datetime.now().strftime("%d-%m-%Y"))+'.\n\n\n'+'Regards,'+'\n'+'ADMS-Data Analytics'+'\n'+'\n'+'\n'+'This is an System Generated E-mail. If you have any queries Please contact to the E-mail id: sairaghava.parisa@spsoft.in'
    return Present_body_mail

start_time =pd.to_datetime(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))

from_address="vineeth.kakarlapudi@spsoft.in"
names_send_dict={"ADMS-Data Analytics":"preethi.polamoni@spsoft.in","Java":"sairaghava.parisa@spsoft.in"}

def get_mail(name):
    return names_send_dict.get(name)

#now_true = datetime.now().strftime('%H:%M:%S')
#send_mail=parameter
while True:
    present_time= pd.to_datetime(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
    counter_time_in_seconds=get_time_diff_in_seconds(start_time,present_time)
    email_send_time_counter_hours=counter_time_in_seconds/60

    #     if datetime.now().strftime('%H:%M:%S')==send_mail:
    if (email_send_time_counter_hours==1):
        manuplations_attendance()
        print('sending_mail')
        time.sleep(1) 

        overall_attendence=pd.read_csv('overall_attendence_final.csv')
        present_day=pd.read_csv('present_day_attdence.csv')

        username = from_address
        password = 'spsoft@123'

############################################ email body and Overall Attachment  #########################################

        for key in names_send_dict.keys():
            #filter_by_name=overall_attendence[overall_attendence['Department']==str(key)]
            #filter_by_name.to_csv("overall_attendence_dept.csv")
            msg = MIMEMultipart()
            msg["From"] = from_address
            to_address =get_mail(key)
            print("Overall",to_address)
            msg["To"] = to_address
            msg["Subject"] = overall_subject()

            body = overall_body()
            body = MIMEText(body) # convert the body to a MIME compatible string
            msg.attach(body)

            attachment_path_1='overall_attendence_final.csv'
            attachment_1 = MIMEBase('application', "octet-stream")
            attachment_1.set_payload(open(attachment_path_1, "rb").read())
            encoders.encode_base64(attachment_1)
            attachment_1.add_header('Content-Disposition', 'attachment', filename= str(datetime.now().strftime("%d-%m-%Y"))+' overall_attendence.csv') 
            msg.attach(attachment_1)

            server = smtplib.SMTP('sg3plvcpnl208309.prod.sin3.secureserver.net')
            server.starttls()
            server.login(username,password)
            server.sendmail(from_address,to_address,msg.as_string())
            server.quit()

########################################### email body and Present Attachment  #########################################

        for key in names_send_dict.keys():
            #filter_by_name1=present_day[present_day['Department']==str(key)]
            #filter_by_name1.to_csv("present_day_dept.csv")
            msg = MIMEMultipart()
            msg["From"] = from_address
            to_address =get_mail(key)
            print("Present Day",to_address)
            msg["To"] = to_address
            msg["Subject"] = present_subject()

            body = present_body()
            body = MIMEText(body) # convert the body to a MIME compatible string
            msg.attach(body)

            attachment_path_2='present_day_attdence.csv'
            attachment_2 = MIMEBase('application', "octet-stream")
            attachment_2.set_payload(open(attachment_path_2, "rb").read())
            encoders.encode_base64(attachment_2)
            attachment_2.add_header('Content-Disposition', 'attachment', filename= str(datetime.now().strftime("%d-%m-%Y"))+' present_day_attdence.csv') 
            msg.attach(attachment_2)

            server = smtplib.SMTP('sg3plvcpnl208309.prod.sin3.secureserver.net')
            server.starttls()
            server.login(username,password)
            server.sendmail(from_address,to_address,msg.as_string())
            server.quit()
        start_time=present_time

C:\Users\spsoft\Anaconda3\lib\site-packages\ipykernel_launcher.py:175: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
C:\Users\spsoft\Anaconda3\lib\site-packages\pandas\core\frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


sending_mail
Overall preethi.polamoni@spsoft.in
Overall sairaghava.parisa@spsoft.in
Present Day preethi.polamoni@spsoft.in
Present Day sairaghava.parisa@spsoft.in


KeyboardInterrupt: 